In [1]:
### Import libraries
import numpy as np
import pandas as pd
from sklearn.linear_model import LogisticRegression
from sklearn.cross_validation import train_test_split
from sklearn import metrics
from sklearn.feature_selection import RFE
from sklearn import cross_validation
from sklearn import preprocessing
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA
%pylab
from sklearn.pipeline import make_pipeline
from scipy.signal import argrelextrema
import statsmodels.discrete.discrete_model as sm

Using matplotlib backend: Qt4Agg
Populating the interactive namespace from numpy and matplotlib


/home/deva/anaconda2/lib/python2.7/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [2]:
df_outliers = pd.read_excel('DistrictTypes.xlsx')

In [3]:
df_outliers.head()

,District,State,Type,DistCode
0,Agra,Uttar Pradesh,4. Large City,146
1,Ahmadabad,Gujarat,1. Metro,474
2,Aizawl,Mizoram,3. Capital,283
3,Ajmer,Rajasthan,4. Large City,119
4,Allahabad,Uttar Pradesh,4. Large City,175


In [4]:
snow_id = df_outliers[df_outliers['Type'] == '5. Snow Clad']['DistCode'].values.tolist()

In [5]:
metro = df_outliers[df_outliers['Type'] == '1. Metro']['DistCode'].values.tolist()

In [6]:
### Import Data Files
df_satData = pd.read_csv('distSatellite_withLabels.csv')

In [7]:
df_satData = df_satData[~(df_satData['101'].isin(snow_id+metro))]

In [8]:
### List of labels
lis_labelsAll = ['MSL_1','MSL_2','MSL_3','MSW_1','MSW_2','MSW_3','CHH_1','CHH_2','CHH_3','FC_1','FC_2','FC_3','BF_1','BF_2','BF_3','EMP_1','EMP_2','EMP_3']

### List of useful Labels
lis_labels= ['EMP_1','EMP_2','EMP_3']

In [9]:
df_satData.head()

,DISTRICT,101,DN_00,DN_01,DN_02,DN_03,DN_04,DN_05,DN_06,DN_07,...,CHH_3,FC_1,FC_2,FC_3,BF_1,BF_2,BF_3,EMP_1,EMP_2,EMP_3
0,Adilabad,532,5237.12,0,0,0.76,1815.67,3332.38,2043.90,1049.05,...,0,1,0,0,1,0,0,0,1,0
1,Agra,146,401.66,0,0,0.00,48.54,451.36,620.77,441.57,...,0,0,0,1,0,0,1,1,0,0
3,Ahmadnagar,522,984.52,0,0,3.12,1756.19,3855.19,2683.53,1989.69,...,0,1,0,0,1,0,0,0,1,0
4,Aizawl,283,3412.59,0,0,0.00,49.31,102.02,42.99,27.32,...,0,0,0,1,0,0,1,0,0,1
5,Ajmer,119,3903.05,0,0,1.61,386.31,1286.76,648.55,490.42,...,0,0,0,1,0,0,1,0,0,1


In [10]:
df_satData.shape

(592, 103)

In [11]:
cols = df_satData.columns.tolist()

In [12]:
cols_new = cols[2:cols.index('MSL_1')]

In [13]:
quantile_list = df_satData[cols_new].quantile(0.95).values.tolist()

In [14]:
for i,quantile_val in enumerate(quantile_list):
    df_satData.loc[df_satData[cols_new[i]] > quantile_val,cols_new[i]] = quantile_val

In [15]:
pure_modis_features = cols[cols.index('Water'):cols.index('MSL_1')]

## Building features from night light data

In [16]:
var = []
for i in range(64):
    var.append('DN_' + str(i).zfill(2))

df_satData['Area'] = 0
for i in var:
    df_satData['Area']=df_satData['Area'] + df_satData[i]
df_satData['Area'] = df_satData['Area'].replace(0,1)

for i in var:
    df_satData['p'+i] = (df_satData[i])/(df_satData['Area']*1.0)

for i,j in enumerate(var):
    df_satData['q'+j] = df_satData[j]*(i+1)

var3 = []
for i in var:
    var3.append('p'+i)
for i in var:
    var3.append('q'+i)

## Appling Pca to matrix formed from features of nightlight data

In [17]:
X = df_satData.loc[:,var3].values

In [18]:
X = preprocessing.scale(X)
pca = PCA()
pca.fit(X)

#pca= PCA(svd_solver='full')
#pca.fit(X, y=None )

#The amount of variance that each PC explains
var= pca.explained_variance_ratio_

#Cumulative Variance explains
var1=np.cumsum(np.round(pca.explained_variance_ratio_, decimals=4)*100)
print(var1)

#Number of PC comprising 95%Variance
n = len(var1)-len([num for num in var1 if num != 0 and num > 85.0])+1
print( "n: 95% Variance: ",n)

[ 43.44  61.91  69.68  73.81  77.19  79.52  81.53  83.45  85.03  86.3
  87.43  88.44  89.21  89.94  90.52  91.05  91.54  92.02  92.41  92.76
  93.08  93.38  93.66  93.92  94.16  94.4   94.63  94.85  95.07  95.27
  95.47  95.66  95.84  96.02  96.19  96.36  96.52  96.68  96.83  96.98
  97.12  97.26  97.39  97.52  97.64  97.76  97.87  97.98  98.08  98.18
  98.28  98.38  98.47  98.56  98.64  98.72  98.8   98.87  98.94  99.01
  99.07  99.13  99.19  99.25  99.3   99.35  99.4   99.45  99.49  99.53
  99.57  99.6   99.63  99.66  99.69  99.71  99.73  99.75  99.77  99.79
  99.81  99.82  99.83  99.84  99.85  99.86  99.87  99.88  99.89  99.9
  99.91  99.92  99.93  99.94  99.95  99.96  99.97  99.97  99.97  99.97
  99.97  99.97  99.97  99.97  99.97  99.97  99.97  99.97  99.97  99.97
  99.97  99.97  99.97  99.97  99.97  99.97  99.97  99.97  99.97  99.97
  99.97  99.97  99.97  99.97  99.97  99.97  99.97  99.97]
('n: 95% Variance: ', 9)


In [19]:
### PCA fit
X=pca.fit_transform(X)
X=X[:,0:n]

In [20]:
n

9

In [21]:
## creating variance table 

In [22]:
df_pcaComp = pd.DataFrame(pca.components_)

In [23]:
pca_idx = ['PCA_'+str(i) for i in range(len(df_pcaComp))]

In [24]:
df_pcaComp.index = pca_idx

In [25]:
df_pcaComp.columns = var3

In [201]:
abs(df_pcaComp[column][:10])/abs(df_pcaComp[:10]).sum(axis=1)

PCA_0    0.007115
PCA_1    0.003367
PCA_2    0.014193
PCA_3    0.007086
PCA_4    0.011504
PCA_5    0.009548
PCA_6    0.004924
PCA_7    0.012804
PCA_8    0.017946
PCA_9    0.010711
dtype: float64

In [202]:
bins_pcaVar = [[0]*10]*7

for column in df_pcaComp:
    bins_pcaVar[(int(column[4:])/10)] += (abs(df_pcaComp[column][:10])/abs(df_pcaComp[:10]).sum(axis=1))

bins_pcaVar[5] += bins_pcaVar[6]

In [203]:
df_bins_pcaVar = pd.DataFrame(bins_pcaVar[:6]).T

In [204]:
df_bins_pcaVar.columns = ['L1','L2','M1','M2','H1','H2']

In [205]:
df_bins_pcaVar

,L1,L2,M1,M2,H1,H2
PCA_0,0.069622,0.182638,0.184764,0.185235,0.168014,0.209727
PCA_1,0.115357,0.118185,0.153546,0.194123,0.184449,0.234339
PCA_2,0.131567,0.287893,0.123489,0.084085,0.107570,0.265397
PCA_3,0.245433,0.173860,0.095888,0.085031,0.114483,0.285305
PCA_4,0.278308,0.154732,0.095628,0.121288,0.113258,0.236787
PCA_5,0.122906,0.132008,0.261625,0.157568,0.111595,0.214298
PCA_6,0.178404,0.122809,0.202930,0.131865,0.216273,0.147720
PCA_7,0.357670,0.173657,0.133177,0.058327,0.081393,0.195776
PCA_8,0.214938,0.154443,0.158474,0.150712,0.102664,0.218769
PCA_9,0.240271,0.193651,0.179624,0.082451,0.149666,0.154337


In [206]:
df_newVarTable = pd.DataFrame(columns=['0','1','2','3','4','5'])

for i in range(10):
    temp = df_bins_pcaVar[i:i+1].sort_values(by=pca_idx[i],axis=1,ascending=False)
    temp_cols = temp.columns.tolist()
    temp_vals = np.round(temp.values[0],2).tolist()
    new_row = [str(temp_vals[j])+','+temp_cols[j] for j in range(len(temp_cols))]
    df_newVarTable.loc[pca_idx[i],:] = new_row
    

In [209]:
df_newVarTable.to_csv('newVarTable.csv')

In [ ]:
temp2 = []
for i in range(n):
    temp = df_pcaComp[i:i+1]
    temp_sorted = temp.sort_values(by=pca_idx[i],axis=1)
    cols = temp_sorted.columns.tolist()
    #temp_sorted[cols[:3]+cols[-3:]]
    temp3 = temp_sorted[cols[:3]+cols[-3:]]
    temp2.append(temp3.columns.tolist())
    temp2.append(temp3.values.tolist()[0])
    temp2.append([float('NaN')]*6)
    df_temp2 = pd.DataFrame(temp2)

In [ ]:
df_temp2.to_csv('variance_table.csv')

In [ ]:
### Creating Dataframe of PCA
col_X = []
for j in range(n):
    col_X.append('PCA_'+str(j))
df_pcaNTL = pd.DataFrame(X,columns=col_X)

In [ ]:
df_pcaNTL['101'] = df_satData['101'].values.tolist()

In [ ]:
df_pcaNTL.head()

## Building features out of Modis Data

In [ ]:
df_satData['Urban and built-up']=df_satData['Urban and built-up']+1
df_satData['CropRatio']=(df_satData['Croplands']+df_satData['Cropland/Natural vegetation mosaic'])/df_satData['Area']
df_satData['AvgUrbanNTL']=df_satData['sum']/df_satData['Urban and built-up']
df_satData['UrbanRatio']= df_satData['Urban and built-up']/df_satData['Area']

df_satData['Natural']= 0
for i in ['Water','Evergreen Needleleaf forest','Evergreen Broadleaf forest','Deciduous Needleleaf forest','Deciduous Broadleaf forest','Mixed forest','Closed shrublands','Open shrublands','Woody savannas','Savannas','Grasslands','Permanent wetlands','Snow and ice']:
    df_satData['Natural']=df_satData['Natural']+df_satData[i]
    
df_satData['CropRemainRatio']=(df_satData['Croplands']+df_satData['Cropland/Natural vegetation mosaic'])/(df_satData['Area']-df_satData['Natural'])
df_satData['UrbanRemainRatio']=df_satData['Urban and built-up']/(df_satData['Area']-df_satData['Natural'])
#df_satData['UrbanRatio']=df_satData['UrbanRatio'].fillna(0)


forest =['Evergreen Broadleaf forest','Deciduous Broadleaf forest','Mixed forest']
df_satData['forest'] = df_satData[forest].sum(axis=1)

grass_shrubs =['Closed shrublands','Open shrublands','Woody savannas','Savannas','Grasslands']
df_satData['grass_shrubs'] = df_satData[grass_shrubs].sum(axis=1)


In [ ]:
modi_area = df_satData[pure_modis_features].sum(axis=1).values.tolist()

In [ ]:
left_area = [df_satData['Area'].values.tolist()[i]-modi_area[i] for i in range(len(df_satData))]

In [ ]:
df_satData['left_area'] = left_area

In [ ]:
modi_var = []
modi_var=['mean','sum','CropRatio', 'AvgUrbanNTL', 'UrbanRatio', 'CropRemainRatio', 'UrbanRemainRatio','Natural']
for i in ['Croplands',
 'Urban and built-up',
 'Cropland/Natural vegetation mosaic',
 'forest',
 'grass_shrubs']:
    df_satData['mod_'+i]=df_satData[i]/df_satData['Area']
    modi_var.append('mod_'+i)
    modi_var.append(i)
modi_var.append('left_area')

## final dataframe from pca version of nl and modi 

In [ ]:
df_modi_nlPca =  pd.merge(df_satData[['101']+modi_var],df_pcaNTL,left_on='101',right_on='101',how='inner')

In [ ]:
df_modi_nlPca.shape

In [ ]:
var = df_modi_nlPca.columns.tolist()[1:]

## finding feature ranking for a particular label

In [ ]:
def featureRank(label):
    model = LogisticRegression()
    selector = RFE(model,1, step=1)

    X = df_modi_nlPca[var].values
    X = preprocessing.scale(X)

    y = df_satData.loc[:,label].values

    selector.fit(X,y)

    selectedIdx = selector.get_support(indices=True)

    featureRanking = selector.ranking_

    ranked_var = pd.DataFrame(featureRanking,var)
    ranked_var = ranked_var.reset_index()
    ranked_var.columns = (['var','rank'])
    ranked_var = ranked_var.sort_values('rank')
    ranked_var.reset_index(inplace=True)
    del ranked_var['index']
    
    return ranked_var

## selecting features to maximize f1_score
#### Assumption: Set of first n features in feature ranking will produce maximum f1_score compared to any other set of n features.

In [ ]:
def selectFeatures(ranked_var,label):
    metric_list = list()
    f1_list = list()
    for i in range(len(ranked_var)):
        var2 = ranked_var.loc[:i,'var'].values.tolist()

        X = df_modi_nlPca.loc[:,var2].values
        y = df_satData.loc[:,label].values

        model = LogisticRegression(class_weight='balanced')
        clf = make_pipeline(preprocessing.StandardScaler(),model)
        predicted = cross_validation.cross_val_predict(clf,X,y, cv=5)

        acc = metrics.accuracy_score(y,predicted)
        recall = metrics.recall_score(y,predicted)
        prec =  metrics.precision_score(y,predicted)

        cm = metrics.confusion_matrix(y, predicted)
        cm_score = float(cm[1][1])/(cm[1][1]+cm[1][0]+cm[0][1])

        f1 = metrics.f1_score(y,predicted)

        metric_list.append([f1,cm_score,acc,recall,prec])
        f1_list.append(f1)

    df_metric = pd.DataFrame(metric_list)
    df_metric.columns =  (['f1','cm','acc','recall','prec'])

    ## finding the global maximum index in f1_list
    global_maxima = f1_list.index(max(f1_list))

    ## plotting f1_score with features selected
    #figure()
    #plt.grid()
    #plt.plot(f1_list)
    #plt.title(label)
    #plt.xlabel('number of variables')
    #plt.ylabel('f1_score')
    
    ## findi  ng indexes of all local maximums

    x = array(f1_list)
    # for local maxima
    local_maxima = argrelextrema(x, np.greater)[0].tolist()

    f1_acceptable = f1_list[global_maxima]-0.01

    i = 0
    while f1_list[i] < f1_acceptable:
        i += 1

    var_final = ranked_var.loc[:i,'var']

    X = df_modi_nlPca.loc[:,var_final].values
    y = df_satData.loc[:,label].values

    model = LogisticRegression()

    fit = model.fit(X,y)

    df_selecedVar = ranked_var.loc[:i].copy(deep=True)
    df_selecedVar.insert(2,'coeff',fit.coef_[0])

    return [df_selecedVar,f1_list[i],df_metric[i:i+1]]

In [ ]:
select_feature = []
score_metric = []
for label in lis_labelsAll:
    ranked_var = featureRank(label)
    feature_and_score = selectFeatures(ranked_var,label)
    select_feature.append(feature_and_score)
    score_metric.append(feature_and_score[2].values[0])

In [ ]:
df_scoreMetric = pd.DataFrame(score_metric,columns = ['f1','cm','acc','recall','prec'], index=lis_labelsAll)

In [ ]:
#df_scoreMetric.to_csv('scoreMetric_pcaModi.csv')

In [ ]:
df_scoreMetric.sort_values(by='f1',ascending=False)

In [ ]:
select_feature[0][0]

In [ ]:
## adding pvalues.... to the list of featues.

In [ ]:
X = df_modi_nlPca.copy(deep=True)
X.insert(1,'intercept',1.0)
cols = X.columns.tolist()

In [ ]:
modi_cols = cols[1:cols.index('PCA_0')]
pca_cols = ['intercept'] + cols[cols.index('PCA_0'):]

In [ ]:
def find_summary(cols,label):
    X1 = X[cols].values
    y = df_satData.loc[:,label].values

    model = sm.Logit(y,X1)
    result = model.fit()
    summary = result.summary()

    df_summary = pd.DataFrame(result.params,result.pvalues)
    df_summary.reset_index(inplace=True)
    df_summary.index = cols
    df_summary.columns = ['pvals','param']
    
    return df_summary

In [ ]:
selectedFeatures_pval = []
for i,label in enumerate(lis_labelsAll):
    df_temp = select_feature[i][0].copy(deep=True)
    df_temp.index = df_temp['var']
    del df_temp['var']
    df_summary = pd.concat([find_summary(modi_cols,label),find_summary(pca_cols,label)])
    df_temp2 = pd.concat([df_temp,df_summary],axis=1,join_axes=[df_temp.index])
    selectedFeatures_pval.append(df_temp2)

In [ ]:
label = 'BF_1'
selectedFeatures_pval[lis_labelsAll.index(label)]